In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
xgb.__version__

/home/tunguz/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.6.0rc1'

In [3]:
!nvidia-smi

Fri Apr 22 15:49:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   33C    P0    59W / 275W |     14MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM-80GB (UUID: GPU-a19c00c3-2832-fe38-1c43-c18db3e909da)
GPU 1: NVIDIA A100-SXM-80GB (UUID: GPU-58b97c92-e879-49d3-85b5-1d9615f10873)
GPU 2: NVIDIA A100-SXM-80GB (UUID: GPU-d21cfed4-2e1a-f313-839c-ea008aca027a)
GPU 3: NVIDIA DGX Display (UUID: GPU-bda05ff2-94b1-c885-24f7-586c8f86b408)
GPU 4: NVIDIA A100-SXM-80GB (UUID: GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac)


In [5]:
train_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-aug-2021/xgtrain.csv')
test_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-aug-2021/xgval.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=["GPU-a19c00c3-2832-fe38-1c43-c18db3e909da",
                                                 "GPU-58b97c92-e879-49d3-85b5-1d9615f10873",
                                                 "GPU-d21cfed4-2e1a-f313-839c-ea008aca027a",
                                                 "GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac"])
client = Client(cluster)

2022-04-22 15:50:53,748 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-22 15:50:53,759 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-22 15:50:53,802 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-22 15:50:53,805 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [8]:
!nvidia-smi

Fri Apr 22 15:51:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   34C    P0    62W / 275W |    428MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

In [9]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)

In [10]:
num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['reg:squarederror']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['rmse']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    rmse = mean_squared_error(test_y, predictions, squared=False)
    
    return rmse

In [11]:
study = optuna.create_study(direction='minimize')

[I 2022-04-22 15:52:03,219] A new study created in memory with name: no-name-738411eb-bee1-404b-8bdc-19bcec8be514


In [12]:
%%time
study.optimize(objective, n_trials=3)

[15:52:13] task [xgboost.dask]:tcp://127.0.0.1:34345 got new rank 0
[15:52:13] task [xgboost.dask]:tcp://127.0.0.1:35441 got new rank 1
[15:52:13] task [xgboost.dask]:tcp://127.0.0.1:46483 got new rank 2
[15:52:13] task [xgboost.dask]:tcp://127.0.0.1:33817 got new rank 3
[I 2022-04-22 15:54:28,099] Trial 0 finished with value: 7.926450749524911 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'lambda': 0.0023513494092192023, 'alpha': 1.8564103853632428, 'colsample_bytree': 0.37139528126242355, 'subsample': 0.5262197416773551, 'learning_rate': 0.03818110712947613, 'max_depth': 20, 'min_child_weight': 133, 'eval_metric': 'rmse'}. Best is trial 0 with value: 7.926450749524911.
[15:54:28] task [xgboost.dask]:tcp://127.0.0.1:33817 got new rank 0
[15:54:28] task [xgboost.dask]:tcp://127.0.0.1:46483 got new rank 1
[15:54:28] task [xgboost.dask]:tcp://127.0.0.1:34345 got new rank 2
[15:54:28] task [xgboost.dask]:tcp://127.0.0.1:35441 got new rank 3
[I 2022-04-22 15:

CPU times: user 29.9 s, sys: 14.3 s, total: 44.2 s
Wall time: 8min 2s


In [13]:
%%time
study.optimize(objective, n_trials=15)

[16:00:37] task [xgboost.dask]:tcp://127.0.0.1:46483 got new rank 0
[16:00:37] task [xgboost.dask]:tcp://127.0.0.1:34345 got new rank 1
[16:00:37] task [xgboost.dask]:tcp://127.0.0.1:35441 got new rank 2
[16:00:37] task [xgboost.dask]:tcp://127.0.0.1:33817 got new rank 3
[I 2022-04-22 16:00:42,989] Trial 3 finished with value: 7.844549097222731 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'lambda': 0.06546086525010585, 'alpha': 0.0020960132641948503, 'colsample_bytree': 0.7404806687689074, 'subsample': 0.6461640004616335, 'learning_rate': 0.014274852508671993, 'max_depth': 3, 'min_child_weight': 55, 'eval_metric': 'rmse'}. Best is trial 3 with value: 7.844549097222731.
[16:00:42] task [xgboost.dask]:tcp://127.0.0.1:33817 got new rank 0
[16:00:42] task [xgboost.dask]:tcp://127.0.0.1:46483 got new rank 1
[16:00:42] task [xgboost.dask]:tcp://127.0.0.1:35441 got new rank 2
[16:00:42] task [xgboost.dask]:tcp://127.0.0.1:34345 got new rank 3
[I 2022-04-22 16:0

CPU times: user 26.4 s, sys: 6.57 s, total: 33 s
Wall time: 10min 48s


/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 3520224 should have a CUDA context assigned to device GPU-58b97c92-e879-49d3-85b5-1d9615f10873, but instead the CUDA context is on device 1. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 3520218 should have a CUDA context assigned to device GPU-a19c00c3-2832-fe38-1c43-c18db3e909da, but instead the CUDA context is on device 0. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warning